In [1]:
# Heat budget terms for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

/Users/weiyangbao/miniconda3/envs/py37/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
sd = 10 # sill depth

state01 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/State700/Hs_' + str(format(sd,'03d')) + '.nc')


state1 = state01.isel(Y=range(1,10), T=range(40,121))

Area = np.empty([50, 9]) # Cross Y direction
Area[:10, :] = 100
Area[10:20, :] = 200
Area[20:30, :] = 300
Area[30:40, :] = 800
Area[40:, :] = 1600

f = 3 # Control volume start index near the ice front
s = 28 # Control volume end index at the sill crest
rho = 1025 # Density

In [3]:
# Read data in T,Z,Y,X
u0 = state1.U.data
u = 0.5 * (u0[:,:,:,:-1] + u0[:,:,:,1:])

w0 = state1.W.data
w = 0.5 * (w0[:,:-1,:,:] + w0[:,1:,:,:])


In [4]:
# Advective flux of potential temperature
Vf = np.sum(Area * u[:,:,:,f].mean(0))
Vs = np.sum(Area * u[:,:,:,s].mean(0))

Vf, Vs

(700.1421100227162, 700.0675454735756)

In [5]:
## Budget terms for different layers

uzx = u.mean(axis=(0,2)) # Z, X
# Time average
wt = w.mean(0) # Z, Y, X
dxdy = 200**2

# Find the interface and vertical flux of theta
def Layer(uzx):
    # Layer interface, zero velocity crossing point
    X = uzx.shape[1]
    Vzx = np.empty(X)
    itf = np.empty(X, dtype=int)
    Vzx[[0,-1]] = 0
    itf[[0,-1]] = 0
    for i in range(1,X-1):
        l = np.where(uzx[:, i] < 0)[-1][0]
        itf[i] = l
        Vzx[i] = np.sum(wt[l,:,i] * dxdy) # Vertical advective flux through the interface
    return (itf, Vzx)

In [6]:
itf, Vzx = Layer(uzx)

Vz = Vzx[f:s+1].sum() 

Vf1 = np.sum(Area[:itf[f],:] * u[:,:itf[f],:,f].mean(0)) 
Vs1 = np.sum(Area[:itf[s],:] * u[:,:itf[s],:,s].mean(0)) 

Vf2 = np.sum(Area[itf[f]:,:] * u[:,itf[f]:,:,f].mean(0))
Vs2 = np.sum(Area[itf[s]:,:] * u[:,itf[s]:,:,s].mean(0))

Vf1, Vs1, Vf2, Vs2, Vz

(3998.9064840599895,
 2168.652582168579,
 -3298.7643740372732,
 -1468.5850366950035,
 -2377.473024189472)